In [2]:
# Load the gams extension
%load_ext gams_magic

# Lights Out

In Tiger Electronic's handheld solitaire game Lights Out, the
player strives to turn out all 25 lights that make up a 5 x 5 grid
of cells.  On each turn, the player is allowed to click on any one cell.
Clicking on a cell activates a switch that causes the states of the
cell and its (edge) neighbours to change from on to off, or from off to on.
Corner cells are considered to have 2 neighbours, edge cells to have
three, and interior cells to have four.

### Formulate and solve an integer program for finding a way to turn out all the lights in as few turns as possible (starting from the state where all lights are on).

Hints: The order in which the cells are clicked
doesn't matter. A cell should not be clicked more than once.

In [16]:
%%gams

set x /1*5/;
set y /1*5/;

* light x,y was clicked
integer variable clicks(x,y);

* number of clicks
variable totclicks;

* integer variable that helps to build the even constraint
integer variable onoff(x,y);

parameter N 'N-way bulb' /1/;
parameter start 'Initial state for each light' /1/;

equation obj;
obj.. totclicks =e= sum((x,y), clicks(x,y));

equation neighbours;
neighbours(x,y)..  (N+1)*onoff(x,y) =e= start + clicks(x-1,y) + clicks(x,y) + clicks(x+1,y) + clicks(x,y-1) + clicks(x,y+1);

model lights /all/;
solve lights min totclicks using MIP;


* number of times each light changes it state 
parameter switch(x,y);
switch(x,y) = clicks.l(x-1,y) + clicks.l(x,y) + clicks.l(x+1,y) + clicks.l(x,y-1) + clicks.l(x,y+1);

display switch, clicks.l, totclicks.l;


,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),15.0,26,51,MIP,CPLEX,0.031


In [18]:
# print the matrix "clicks" and totclicks here
%gams_pull -d clicks totclicks
#display(clicks)
clicks.columns = ['i','j','l','m','lo','up','s']
clicks = clicks.pivot(index='i',columns='j',values='l')
clicks.index.names = [None]
clicks.columns.names = [None]
display(clicks,totclicks)


,1,2,3,4,5
1,0.0,0.0,0.0,1.0,1.0
2,1.0,1.0,0.0,1.0,1.0
3,1.0,1.0,1.0,0.0,0.0
4,0.0,1.0,1.0,1.0,0.0
5,1.0,0.0,1.0,1.0,0.0


,level,marginal,lower,upper,scale
0,15.0,0.0,-inf,inf,1.0


What if each cell has a three-way bulb?
(Repeatedly clicking on a single three way bulb changes its state from
off to low, from low to medium, from medium to high, from high to off, and
so on.)  

### Which is easiest (a) turning off all the lights when they're all on their high setting, (b) turning them off when they're all on medium, or (c) turning them off when they're all on low?

In [36]:
%%gams

N = 3;
parameter clicks_a, clicks_b, clicks_c;
parameter totclicks_a, totclicks_b, totclicks_c;

start = 3;
clicks.l(x,y) = 0;
solve lights min totclicks using MIP;
clicks_a(x,y) = clicks.l(x,y);
totclicks_a = totclicks.l;

start = 2;
clicks.l(x,y) = 0;
solve lights min totclicks using MIP;
clicks_b(x,y) = clicks.l(x,y);
totclicks_b = totclicks.l;

start = 1;
clicks.l(x,y) = 0;
solve lights min totclicks using MIP;
clicks_c(x,y) = clicks.l(x,y);
totclicks_c = totclicks.l;


,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Integer (8),39.0,26,51,MIP,CPLEX,1.062
1,Normal (1),Integer (8),30.0,26,51,MIP,CPLEX,30.813
2,Normal (1),Integer (8),29.0,26,51,MIP,CPLEX,0.094


In [37]:
# print the matrix "clicks" totclicks here in all three cases
%gams_pull -d clicks_a clicks_b clicks_c

def get_matrix(clicks):
    clicks.columns = ['x','y','value']
    clicks = clicks.pivot(index='x',columns='y',values='value')
    clicks.index.names = [None]
    clicks.columns.names = [None]
    clicks = clicks.fillna(0)
    display(clicks)

get_matrix(clicks_a)
get_matrix(clicks_b)
get_matrix(clicks_c)


,1,2,3,4,5
1,0.0,0.0,2.0,1.0,3.0
2,1.0,3.0,2.0,3.0,1.0
3,1.0,3.0,3.0,2.0,2.0
4,0.0,3.0,3.0,3.0,0.0
5,1.0,0.0,1.0,1.0,0.0


,1,2,3,4,5
1,0.0,0.0,0.0,2.0,2.0
2,2.0,2.0,0.0,2.0,2.0
3,2.0,2.0,2.0,0.0,0.0
4,0.0,2.0,2.0,2.0,0.0
5,2.0,0.0,2.0,2.0,0.0


,1,2,3,4,5
1,3.0,2.0,1.0,1.0,0.0
2,2.0,1.0,3.0,1.0,2.0
3,1.0,3.0,1.0,0.0,0.0
4,1.0,1.0,0.0,1.0,1.0
5,0.0,2.0,0.0,1.0,1.0


In [50]:
%gams_pull totclicks_a totclicks_b totclicks_c
print('totclicks_a = ' + str(totclicks_a[0]))
print('totclicks_b = ' + str(totclicks_b[0]))
print('totclicks_c = ' + str(totclicks_c[0]))
print('c is the easiest one')

totclicks_a = 39.0
totclicks_b = 30.0
totclicks_c = 29.0
c is the easiest one
